In [1]:
#%% Imports:
import matplotlib
matplotlib.use('MACOSX')

from math import sqrt
import os
import numpy as np
import mne
#import matplotlib #doesnt it import through mne already?
from copy import deepcopy

#Open data:
#sample_data_folder = mne.datasets.sample.data_path()
#kath_raw_file2 = "/Users/jenya/Documents/Oldenburg and university/Job Uni Rieger lab/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif"
#kath_raw_file = os.path.join('Katharinas_Data','sub_HT05ND16', '210811', 'mikado-1.fif')
kath_raw_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif'
print(kath_raw_file)
#print(kath_raw_file2)                                   
raw = mne.io.read_raw_fif(kath_raw_file)
#raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

#Print info about the data:
print(raw)
print(raw.info)
raw

#crop the data to calculate faster

raw_cropped = raw.copy()
raw_cropped.crop(0, 300) #(first 5 min)

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif

/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_28406/42220836.py:19: RuntimeWarning: This filename (/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Katharinas_Data/sub_HT05ND16/210811/mikado-1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(kath_raw_file)


<Raw | mikado-1.fif, 319 x 3390000 (3390.0 s), ~6.6 MB, data not loaded>
<Info | 24 non-empty values
 acq_pars: ACQactiveGround 1 ACQch.BIO001.gain 2000 ACQch.BIO001.highpass ...
 bads: []
 ch_names: IASX+, IASX-, IASY+, IASY-, IASZ+, IASZ-, IAS_DX, IAS_DY, IAS_X, ...
 chs: 11 IAS, 102 Magnetometers, 204 Gradiometers, 1 Stimulus, 1 SYST
 custom_ref_applied: False
 description: TRIUX system at Oldenburg_3127
 dev_head_t: MEG device -> head transform
 dig: 356 items (3 Cardinal, 5 HPI, 348 Extra)
 events: 1 item (list)
 experimenter: Meg User (meg)
 file_id: 4 items (dict)
 gantry_angle: 68.0
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 hpi_subsystem: 3 items (dict)
 line_freq: 50.0
 lowpass: 330.0 Hz
 meas_date: 2021-08-11 12:05:17 UTC
 meas_id: 4 items (dict)
 nchan: 319
 proj_id: 1 item (ndarray)
 proj_name: mikado
 projs: magn8_iasoff_68deg.fif : PCA-v1: off, magn8_iasoff_68deg.fif : ...
 sfreq: 1000.0 Hz
 subject_info: 6 items (dict)
>


<Raw | mikado-1.fif, 319 x 300001 (300.0 s), ~6.6 MB, data not loaded>

In [2]:
#%% Find magnetometers and gradiometers:
# unit t - magenetometer. unit M_T - gradiometer. (in this set name will end with 1 for magnet, 
# with 2and3 for grad.)

mags = []
grads = []

for i, chs in enumerate(raw_cropped.info['chs']):

    if str(chs['unit']).endswith('UNIT_T)'):
        mags.append((chs['ch_name'], i))
    elif str(chs['unit']).endswith('UNIT_T_M)'):
        grads.append((chs['ch_name'], i))

In [7]:
#epoch channels separately
events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)

picks_grad = mne.pick_types(raw_cropped.info, meg='grad', eeg=False, eog=False, stim=False)
picks_magn = mne.pick_types(raw_cropped.info, meg='mag', eeg=False, eog=False, stim=False)

epochs_mags = mne.Epochs(raw_cropped, events, picks=picks_magn, tmin=-0.2, tmax=1, preload=True, baseline = None)
epochs_grads = mne.Epochs(raw_cropped, events, picks=picks_grad, tmin=-0.2, tmax=1, preload=True, baseline = None)

#Present epochs as data frame - separately for mags and grads

df_epochs_mags = epochs_mags.to_data_frame(time_format=None, scalings=dict(mag=1))

df_epochs_grads = epochs_grads.to_data_frame(time_format=None, scalings=dict(grad=1))

Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
8 events found
Event IDs: [ 9 19 20 21 22]
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
8 projection items activated
Loading data for 8 events and 1201 original time points ...
0 bad epochs dropped


/var/folders/h2/khhmb4p510vg63hbv0qkftt80000gs/T/ipykernel_28406/3198487672.py:2: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(raw_cropped, stim_channel='STI101', min_duration=1.2)


In [8]:
df_epochs_mags

,time,condition,epoch,MEG0111,MEG0121,MEG0131,MEG0141,MEG0211,MEG0221,MEG0231,...,MEG2431,MEG2441,MEG2511,MEG2521,MEG2531,MEG2541,MEG2611,MEG2621,MEG2631,MEG2641
0,-0.200,19,0,-1.052550e-13,-1.010281e-13,-1.854901e-13,-2.046335e-14,-8.733612e-14,-4.008955e-14,1.126569e-15,...,1.811787e-13,1.040461e-13,1.232545e-13,2.749943e-14,6.741374e-14,-1.316768e-13,1.138710e-13,-1.050413e-13,-1.547003e-13,-7.218119e-14
1,-0.199,19,0,-7.176393e-14,-6.989817e-14,-1.322054e-13,-1.235434e-13,-1.221540e-13,3.185761e-14,2.634028e-15,...,1.737091e-13,1.346291e-13,2.342058e-13,4.546008e-14,1.030077e-13,-5.103772e-16,1.328093e-13,-1.015670e-13,-8.445258e-14,4.878084e-14
2,-0.198,19,0,-5.981908e-14,-6.188627e-14,-1.186776e-13,-5.027599e-14,-6.281086e-14,1.163183e-13,-3.100246e-14,...,9.176812e-14,1.223074e-13,2.089508e-13,2.854316e-14,1.103020e-13,1.341773e-13,1.430854e-13,-1.438513e-13,-1.631808e-14,1.435712e-13
3,-0.197,19,0,-2.597581e-14,-2.902860e-15,-1.783016e-13,-7.686604e-14,4.132941e-14,6.415045e-14,5.314975e-14,...,5.581505e-14,4.516074e-14,1.164921e-13,-9.217439e-14,-2.699964e-15,9.865452e-14,1.257271e-13,-9.663197e-14,-9.371890e-14,2.456428e-15
4,-0.196,19,0,1.149084e-14,-6.599419e-15,-1.539149e-13,-4.878048e-14,3.145575e-14,-1.986044e-14,4.528828e-14,...,6.443225e-14,5.514481e-14,1.383022e-13,-2.588048e-13,-4.438000e-14,6.780823e-15,1.862074e-14,-7.515577e-15,-1.693029e-13,-1.668383e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9603,0.996,20,7,-6.228715e-14,1.220845e-13,-6.381596e-14,1.636484e-14,8.478514e-14,8.649018e-14,3.057749e-14,...,-3.157250e-14,1.388619e-14,4.651379e-14,-1.869224e-13,-1.020158e-13,5.429833e-14,1.135358e-13,-2.424900e-13,-1.825679e-13,4.224788e-14
9604,0.997,20,7,3.919976e-14,5.097377e-14,-2.429860e-13,-6.197932e-14,-5.329120e-14,6.942176e-14,-1.226127e-13,...,4.960003e-14,7.547381e-14,2.273109e-14,-1.452702e-13,-1.838126e-13,-5.108490e-14,1.925601e-13,-1.872688e-13,-1.486202e-13,-2.174185e-14
9605,0.998,20,7,-3.304769e-14,-1.928448e-13,-9.240115e-14,-5.516289e-14,4.716283e-14,5.531164e-14,-9.274151e-14,...,8.943191e-14,8.401663e-14,-1.287295e-13,-7.350086e-14,-2.009147e-13,-2.613605e-13,1.655408e-13,-4.122324e-14,-8.367578e-14,-3.944463e-15
9606,0.999,20,7,-8.946135e-14,-2.687745e-13,5.924864e-14,9.514944e-15,4.557821e-14,6.935153e-14,1.416351e-13,...,4.536493e-14,1.342086e-13,-7.798118e-14,-1.667561e-14,-1.547291e-13,-2.521886e-13,8.019622e-14,-1.370924e-13,-1.883981e-13,5.192944e-14


In [9]:
df_epochs_mags['MEG0111'][0]

-1.0525499935987298e-13